# Assignment for Module 8 Checkpoint 5

### Begin init

In [2]:
import psycopg2
from sqlalchemy import create_engine
import requests as rq
import pandas as pd

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'dvdrentals'
db_connection = 'postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db)


## 1. How many movies are released for each rating?

In [91]:
query = '''
SELECT rating, COUNT(*) AS total_count FROM film
GROUP BY rating
;'''

rating_count = pd.read_sql(query, db_connection)
rating_count

,rating,total_count
0,R,195
1,NC-17,210
2,PG,194
3,PG-13,223
4,G,178


## 2. What is the average rental duration for each rating?

In [92]:
query = '''
SELECT rating, ROUND(AVG(rental_duration), 2) AS avg_rent_duration FROM film
GROUP BY rating
;'''

avg_rent_duration = pd.read_sql(query, db_connection)
avg_rent_duration

,rating,avg_rent_duration
0,R,4.77
1,NC-17,5.14
2,PG,5.08
3,PG-13,5.05
4,G,4.84


## 3. What is the mean movie length? Calculate this by defining a function.

In [100]:
def mean_movie_length(param, db_con):
    query = '''
    SELECT {pr}, ROUND(AVG(length), 2) AS mean_duration FROM film
    GROUP BY {pr};'''.format(pr=param)
    return pd.read_sql(query, db_con)

mean_duration_rating = mean_movie_length('rating', db_connection)
mean_duration_release = mean_movie_length('release_year', db_connection)
mean_duration_rate = mean_movie_length('rental_rate', db_connection)
print(mean_duration_rate, '\n')
print(mean_duration_rating, '\n')
print(mean_duration_release, '\n')

   rental_rate  mean_duration
0         2.99         117.19
1         4.99         115.82
2         0.99         112.91 

  rating  mean_duration
0      R         118.66
1  NC-17         113.23
2     PG         112.01
3  PG-13         120.44
4      G         111.05 

   release_year  mean_duration
0          2006         115.27 



## 4. What is the median movie length? Calculate this by defining a function.

In [101]:
db_engine = create_engine(db_connection)

query = '''
CREATE OR REPLACE FUNCTION _final_median(NUMERIC[])
    RETURNS NUMERIC AS
$$
    SELECT AVG(val)
    FROM (
        SELECT val
        FROM unnest($1) val
        ORDER BY 1
        LIMIT  2 - MOD(array_upper($1, 1), 2)
        OFFSET CEIL(array_upper($1, 1) / 2.0) - 1
    ) sub;
$$
LANGUAGE 'sql' IMMUTABLE;

CREATE AGGREGATE median(NUMERIC) (
    SFUNC=array_append,
    STYPE=NUMERIC[],
    FINALFUNC=_final_median,
    INITCOND='{}'
);'''

db_engine.connect().execute(query)

ProgrammingError: (psycopg2.errors.DuplicateFunction) function "median" already exists with same argument types

[SQL: 
CREATE OR REPLACE FUNCTION _final_median(NUMERIC[])
    RETURNS NUMERIC AS
$$
    SELECT AVG(val)
    FROM (
        SELECT val
        FROM unnest($1) val
        ORDER BY 1
        LIMIT  2 - MOD(array_upper($1, 1), 2)
        OFFSET CEIL(array_upper($1, 1) / 2.0) - 1
    ) sub;
$$
LANGUAGE 'sql' IMMUTABLE;

CREATE AGGREGATE median(NUMERIC) (
    SFUNC=array_append,
    STYPE=NUMERIC[],
    FINALFUNC=_final_median,
    INITCOND='{}'
);]
(Background on this error at: http://sqlalche.me/e/f405)

In [6]:
def median_movie_length(param, db_con):
    query = '''
    SELECT {pr}, MEDIAN(length) AS median_duration FROM film
    GROUP BY {pr};'''.format(pr=param)
    return pd.read_sql(query, db_con)

median_duration_rating = median_movie_length('rating', db_connection)
median_duration_release = median_movie_length('release_year', db_connection)
median_duration_rate = median_movie_length('rental_rate', db_connection)
print(median_duration_rate, '\n')
print(median_duration_rating, '\n')
print(median_duration_release, '\n')

   rental_rate  median_duration
0         2.99            116.0
1         4.99            116.5
2         0.99            111.0 

  rating  median_duration
0      R            115.0
1  NC-17            112.5
2     PG            113.5
3  PG-13            125.0
4      G            107.5 

   release_year  median_duration
0          2006            114.0 



## 5. Calculate the standard deviation of the movie lengths. Calculate this by defining a function.

In [8]:
def stddev_movie_length(param, db_con):
    query = '''
    SELECT {pr}, ROUND(STDDEV(length), 2) AS stddev_duration FROM film
    GROUP BY {pr};'''.format(pr=param)
    return pd.read_sql(query, db_con)

stddev_duration_rating = stddev_movie_length('rating', db_connection)
stddev_duration_release = stddev_movie_length('release_year', db_connection)
stddev_duration_rate = stddev_movie_length('rental_rate', db_connection)
print(stddev_duration_rate, '\n')
print(stddev_duration_rating, '\n')
print(stddev_duration_release, '\n')

   rental_rate  stddev_duration
0         2.99            39.92
1         4.99            40.80
2         0.99            40.54 

  rating  stddev_duration
0      R            38.51
1  NC-17            40.82
2     PG            39.26
3  PG-13            41.17
4      G            41.77 

   release_year  stddev_duration
0          2006            40.43 

